In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
plotly.__version__

'3.8.1'

In [2]:
res = pd.read_csv('features_generadas.csv')
display(res.head())
print(res.columns)

,user_id,user_id.1,bot,Cat,tweetsCount,semanas,lun,mar,mie,jue,...,num_mentions_mean,num_mentions_median,num_mentions_std,num_mentions_fq,num_mentions_tq,reply_count_mean,reply_count_median,reply_count_std,reply_count_fq,reply_count_tq
0,678033.0,678033.0,False,Humans,300,4,12.666667,17.666667,8.333333,19.333333,...,9313.304721,10000.0,8781.379387,0.0,10000.0,0.0,0.0,0.0,0.0,0.0
1,722623.0,722623.0,False,Humans,300,10,4.444444,4.000000,3.888889,4.600000,...,5591.836735,0.0,6086.528956,0.0,10000.0,0.0,0.0,0.0,0.0,0.0
2,755116.0,755116.0,False,Humans,300,5,4.666667,21.000000,7.000000,28.000000,...,7824.561404,10000.0,8609.532072,0.0,10000.0,0.0,0.0,0.0,0.0,0.0
3,755746.0,755746.0,False,Humans,300,10,4.333333,3.900000,4.400000,5.875000,...,6236.162362,10000.0,6073.892403,0.0,10000.0,0.0,0.0,0.0,0.0,0.0
4,785080.0,785080.0,False,Humans,300,3,14.333333,36.000000,27.333333,3.333333,...,6996.336996,0.0,9142.860510,0.0,10000.0,0.0,0.0,0.0,0.0,0.0


Index(['user_id', 'user_id.1', 'bot', 'Cat', 'tweetsCount', 'semanas', 'lun',
       'mar', 'mie', 'jue', 'vie', 'sab', 'dom', 'times_mean', 'times_median',
       'times_std', 'times_fq', 'times_tq', 'timesRT_mean', 'timesRT_median',
       'timesRT_std', 'timesRT_fq', 'timesRT_tq', 'timesUser_mean',
       'timesUser_median', 'timesUser_std', 'timesUser_fq', 'timesUser_tq',
       'favorite_count_mean', 'favorite_count_median', 'favorite_count_std',
       'favorite_count_fq', 'favorite_count_tq', 'retweet_count_mean',
       'retweet_count_median', 'retweet_count_std', 'retweet_count_fq',
       'retweet_count_tq', 'num_mentions_mean', 'num_mentions_median',
       'num_mentions_std', 'num_mentions_fq', 'num_mentions_tq',
       'reply_count_mean', 'reply_count_median', 'reply_count_std',
       'reply_count_fq', 'reply_count_tq'],
      dtype='object')


In [3]:
def getmarker(usr, sizeVar = False):
    col = False
    if(sizeVar != False):
        col = usr[sizeVar]
        col -= col.min()  # equivalent to df = df - df.min()
        col /= col.quantile(0.3)  # equivalent to df = df / df.max()
        col += [4 for x in range(len(col))]
    return dict(
        size = (lambda: 4 if sizeVar == False else usr[sizeVar])(),
        opacity=0.3
    )

def plotearVars(variables, sizeVar=False):
    colors = {'Humans':'red', 'SocialSpambot1':'blue', 'SocialSpambot2':'green', 'SocialSpambot3':'black'}
    resH = res[res.Cat=='Humans']
    res1 = res[res.Cat=='SocialSpambot1']
    res2 = res[res.Cat=='SocialSpambot2']
    res3 = res[res.Cat=='SocialSpambot3']
    
    trace1 = go.Scatter3d(
        name='Humans',
        x=resH[variables[0][0]],
        y=resH[variables[1][0]],
        z=resH[variables[2][0]],
        mode='markers',
        marker=getmarker(resH, sizeVar)
    )
    trace2 = go.Scatter3d(
        name='SocialSpambot1',
        x=res1[variables[0][0]],
        y=res1[variables[1][0]],
        z=res1[variables[2][0]],
        mode='markers',
        marker=getmarker(res1, sizeVar)
    )
    trace3 = go.Scatter3d(
        name='SocialSpambot2',
        x=res2[variables[0][0]],
        y=res2[variables[1][0]],
        z=res2[variables[2][0]],
        mode='markers',
        marker=getmarker(res2, sizeVar)
    )
    trace4 = go.Scatter3d(
        name='SocialSpambot3',
        x=res3[variables[0][0]],
        y=res3[variables[1][0]],
        z=res3[variables[2][0]],
        mode='markers',
        marker=getmarker(res3, sizeVar)
    )
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=30
        ),
        scene = dict(
            aspectmode='cube',
            xaxis = dict(
                range = (lambda: None if len(variables[0])==1 else variables[0][1])(),
                title=variables[0][0],
            ),
            yaxis = dict(
                range = (lambda: None if len(variables[1])==1 else variables[1][1])(),
                title=variables[1][0],
            ),
            zaxis = dict(
                range = (lambda: None if len(variables[2])==1 else variables[2][1])(),
                title= variables[2][0],
            ),
        )
    )
    fig = go.Figure(data=[trace1,trace2,trace3,trace4], layout=layout)
    iplot(fig)
    return trace1, fig, layout

In [4]:
trace, fig, layout = plotearVars([
    ['times_median', [0,10000]],
    ['lun', [0,100]],
    ['timesUser_tq'],
])
# plotly.offline.plot(fig, filename='grafico1.html')

In [5]:
trace, fig, layout = plotearVars([
    ['times_tq'],
    ['favorite_count_mean', [0, 60000]],
    ['timesUser_tq'],
])
# plotly.offline.plot(fig, filename='grafico2.html')

In [11]:
trace, fig, layout = plotearVars([
    ['times_tq'],
    ['timesUser_tq', [0, 100000]],
    ['num_mentions_std', [0, 15000]],
])
# plotly.offline.plot(fig, filename='grafico3.html')

In [7]:
trace, fig, layout = plotearVars([
    ['vie', [0,100]],
    ['lun', [0,100]],
    ['dom', [0,100]],
], 'semanas')

ValueError: 
    Invalid element(s) received for the 'size' property of scatter3d.marker
        Invalid elements include: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

In [9]:
plotearVars([
    ['times_tq'],
    ['timesUser_tq', [0, 20000]],
    ['timesRT_tq'],
])
print()